In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("Data_Final")
data.head()

,Unnamed: 0,User_id,Bus_id,Star,Useful,Cool,Funny,Review,State,City,Bus_Ave_Star,User_Review_count,User_Useful_count,User_Funny_count,User_Cool_count,Elite,User_Fans,Users_Ave_Star
0,0,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",CA,Goleta,4.0,922,1687,694,1070,"2015,2016,2017,2018,2019,20,20,2021",51,4.20
1,1,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,0,0,0,We were a bit weary about trying the Shellfish...,CA,Santa Barbara,4.0,338,800,144,353,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",30,4.12
2,2,Uk3X2AypU8AqvcYEVf7s6Q,eL4lyE7LNoXEMvpcJ8WNVw,3.0,5,0,0,This was a tough one! On the merits of the w...,CA,Santa Barbara,4.0,431,2126,1245,1476,"2012,2013,2014,2015,2016,2017,2018,2019,20,20,...",76,4.05
3,3,LcqNuhqaYt5ekKzaRirmIg,SZU9c8V2GuREDN5KgyHFJw,5.0,2,1,0,"I love trying fresh seafood on piers, wharfs a...",CA,Santa Barbara,4.0,258,452,125,183,"2015,2016,2017",37,3.99
4,4,gasLVm0KRwrVhPGRcqATjw,CHh0ZFrQcsk4boOItr2Zuw,4.0,1,2,0,I stopped in because I was hungry for some sna...,CA,Carpinteria,3.0,1638,2658,933,2297,"2014,2015,2016,2017,2018,2019,20,20,2021",105,3.79


In [6]:
data.shape

(53845, 18)

In [11]:
data["City"].unique()

array(['Goleta', 'Santa Barbara', 'Carpinteria', 'Montecito',
       'Isla Vista', 'Santa Barbara ', 'Truckee', 'Summerland',
       'Port Hueneme', 'West Hill', 'Los Angeles', 'Mission Canyon',
       'Sparks', 'Kings Beach', 'Cerritos', 'Santa Clara', 'Reno',
       'Real Goleta', 'Aliso Viejo', 'Santa Barbara & Ventura Counties',
       'South Lake Tahoe', 'Santa Barbra', 'Santa  Barbara', 'Salinas',
       'Santa Maria'], dtype=object)

In [13]:
data.groupby("City")["Star"].count()

City
Aliso Viejo                             1
Carpinteria                          2557
Cerritos                               10
Goleta                               6009
Isla Vista                           1425
Kings Beach                             1
Los Angeles                             3
Mission Canyon                         10
Montecito                             848
Port Hueneme                            1
Real Goleta                             4
Reno                                    1
Salinas                                 2
Santa  Barbara                         20
Santa Barbara                       42501
Santa Barbara                           6
Santa Barbara & Ventura Counties        1
Santa Barbra                            4
Santa Clara                            15
Santa Maria                             3
South Lake Tahoe                        1
Sparks                                  2
Summerland                            406
Truckee                      